<a href="https://colab.research.google.com/github/JRamos84/deteccion_fraude/blob/main/notebooks/02_Data_Cleaning_and_Preprocessing_selection_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 02 Data Cleaning and Preprocessing
**Objetivo**: Limpiar y preprocesar los datos para que estén listos para el modelado.
**Contenido**:
- Manejo de valores nulos.
- Creación de nuevas características (feature engineering).
- Conversión de tipos de datos.
- Codificación de variables categóricas.
- Normalización y estandarización de las variables.


In [1]:
# Librerías
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from scipy import stats
import datetime
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.pipeline import Pipeline
import joblib
warnings.filterwarnings('ignore')

In [2]:
def reduce_mem_usage(df, verbose=True):
    """
    Reduce el uso de memoria de un DataFrame de pandas cambiando los tipos de datos de las columnas numéricas
    a tipos más eficientes sin perder información.

    Parameters:
    df (pd.DataFrame): El DataFrame a optimizar.
    verbose (bool): Si es True, imprime el uso de memoria antes y después de la optimización.

    Returns:
    pd.DataFrame: El DataFrame optimizado.
    """
    # Obtener el uso inicial de memoria
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            # Optimización para tipos de datos enteros
            if pd.api.types.is_integer_dtype(df[col]):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # Optimización para tipos de datos flotantes
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # Obtener el uso final de memoria
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Mem. usage decreased to {end_mem:.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')

    return df

In [3]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'

Mounted at /content/drive


In [4]:

## Cargamos los dataset de train_transaction y train_identity
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv')
df_identity_train = pd.read_csv(data_dir + '/train_identity.csv')
## Reducimos los tamaños de los dataset cambiando los tipos de los datos de las columnas
df_transaction_train = reduce_mem_usage(df_transaction_train)
df_identity_train = reduce_mem_usage(df_identity_train)

Mem. usage decreased to 867.89 Mb (58.7% reduction)
Mem. usage decreased to 138.38 Mb (12.2% reduction)


In [5]:

# Combinar los datasets
data = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left')


In [6]:
dataset = data.copy()

## Consideración principales del análisis de EDA

### Manejo de valores nulos

- Usaremos las columnas que no superen el 50 de los valores nulos

In [7]:
null_percentages = (dataset.isnull().sum() / len(dataset)) * 100
null_percentages[null_percentages < 50].sort_values(ascending=False)

M4               47.658753
D2               47.549192
V1               47.293494
V10              47.293494
D11              47.293494
                   ...    
C12               0.000000
C13               0.000000
C14               0.000000
isFraud           0.000000
TransactionID     0.000000
Length: 220, dtype: float64

In [8]:
col_util = null_percentages[null_percentages < 20].index.to_list()

In [9]:
dataset = dataset[col_util]
dataset.shape

(590540, 182)

In [10]:
## Eliminamos las filas nulas
dataset.dropna(inplace=True)
dataset.shape

(346886, 182)

- La cantidad de registro dismuyo considerablemente luego de la elimación de las columnas y las filas con datos nulos

### Rango de TransactionAMT y Aplicación del Logaritmo

- Basado en el análisis EDA elinamos los registro con valores en TransactionAMT superior a 5192

In [11]:
dataset = dataset[dataset['TransactionAmt'] < 5192]

In [12]:
dataset['TransactionAmt'] = np.log(dataset['TransactionAmt'])


####Convertiendo 'TransactionDT' en unidades de tiempo

In [13]:

# Suponiendo que dataset tiene las columnas 'TransactionDT'
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")

# Convertir 'TransactionDT' a formato datetime
dataset['Date'] = dataset['TransactionDT'].apply(lambda x: startdate + datetime.timedelta(seconds=x))

# Verificar y asegurarse de que 'Date' sea de tipo datetime
dataset['Date'] = pd.to_datetime(dataset['Date'])

# Crear un DataFrame con las nuevas columnas a agregar
new_columns = pd.DataFrame({
    '_Weekdays': dataset['Date'].dt.dayofweek,
    '_Hours': dataset['Date'].dt.hour,
    '_Days': dataset['Date'].dt.day
})

# Unir todas las columnas al DataFrame original
dataset = pd.concat([dataset, new_columns], axis=1)

# Convertir las nuevas columnas a tipo 'object' si es necesario
dataset['_Weekdays'] = dataset['_Weekdays'].astype('object')
dataset['_Hours'] = dataset['_Hours'].astype('object')
dataset['_Days'] = dataset['_Days'].astype('object')

# Eliminar las columnas innecesarias
dataset.drop(['Date', 'TransactionDT'], axis=1, inplace=True)

# Mostrar los primeros 3 registros del DataFrame modificado
print(dataset.head(3))

   TransactionID  isFraud  TransactionAmt ProductCD  card1  card2  card3  \
1        2987001        0        3.367188         W   2755  404.0  150.0   
2        2987002        0        4.078125         W   4663  490.0  150.0   
3        2987003        0        3.912109         W  18132  567.0  150.0   

        card4  card5   card6  ...  V315  V316    V317   V318  V319  V320  \
1  mastercard  102.0  credit  ...   0.0   0.0     0.0    0.0   0.0   0.0   
2        visa  166.0   debit  ...   0.0   0.0     0.0    0.0   0.0   0.0   
3  mastercard  117.0   debit  ...   0.0  50.0  1404.0  790.0   0.0   0.0   

   V321  _Weekdays  _Hours  _Days  
1   0.0          5       0      2  
2   0.0          5       0      2  
3   0.0          5       0      2  

[3 rows x 184 columns]


### Mapping emails

In [14]:
#https://www.kaggle.com/code/kabure/extensive-eda-and-modeling-xgb-hyperopt?scriptVersionId=18427966&cellId=116
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum',
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft',
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft',
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other',
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink',
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other',
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo',
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other',
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other',
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other',
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
#for c in ['P_emaildomain', 'R_emaildomain']:
for c in ['P_emaildomain']:
    dataset[c + '_bin'] = dataset[c].map(emails)
    dataset[c + '_suffix'] = dataset[c].map(lambda x: str(x).split('.')[-1])
    dataset[c + '_suffix'] = dataset[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

## Elimaamos las columnas 'P_emaildomain', 'R_emaildomain'
dataset.drop(['P_emaildomain'], axis=1, inplace=True)
### IMPORTANTE  'R_emaildomain' fue sacada del analisis debido a que tiene un elavado cantidad de valores nulos


## Separamos datos características (X) y variable objetivo (y)

In [15]:
X = dataset.drop('isFraud', axis=1)  # features
y = dataset['isFraud']  # target

## División de los Datos en Conjuntos de Entrenamiento y Prueba

### Criterios para la División de Datos

#### Entrenamiento (Training Set):
- Representa el 70% del conjunto de datos.
- Se utiliza para ajustar el modelo.

#### Validación (Validation Set):
- Representa el 15% del conjunto de datos.
- Se utiliza para ajustar los hiperparámetros y prevenir el sobreajuste.

#### Prueba (Test Set):
- Representa el 15% del conjunto de datos.
- Se utiliza para evaluar el rendimiento final del modelo.


In [16]:
X.shape, y.shape

((346874, 184), (346874,))

In [17]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y conjunto temporal (que se dividirá en validación y prueba)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Dividir el conjunto temporal en conjunto de validación y conjunto de prueba
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Mostrar las formas de los conjuntos de datos para asegurarse de que la estratificación se realizó correctamente
print("Forma del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Forma del conjunto de validación:", X_val.shape, y_val.shape)
print("Forma del conjunto de prueba:", X_test.shape, y_test.shape)

Forma del conjunto de entrenamiento: (242811, 184) (242811,)
Forma del conjunto de validación: (52031, 184) (52031,)
Forma del conjunto de prueba: (52032, 184) (52032,)


In [18]:
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

### Codificación de variables categóricas.


In [19]:
X_train.shape, X_val.shape, X_test.shape

((242811, 184), (52031, 184), (52032, 184))

In [20]:
X_train.select_dtypes(include=['object']).nunique()

ProductCD                4
card4                    4
card6                    3
_Weekdays                7
_Hours                  24
_Days                   31
P_emaildomain_bin        9
P_emaildomain_suffix     7
dtype: int64

In [21]:
X_train.dtypes

TransactionID             int32
TransactionAmt          float16
ProductCD                object
card1                     int16
card2                   float16
                         ...   
_Weekdays                object
_Hours                   object
_Days                    object
P_emaildomain_bin        object
P_emaildomain_suffix     object
Length: 184, dtype: object

In [22]:
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(X_train, X_val,X_test,cat_cols):
  X_train = X_train.copy()
  X_val = X_val.copy()
  X_test = X_test.copy()
  for col in cat_cols:
      #print('aqui')
      if X_train[col].dtype=='object' or X_val[col].dtype=='object' or X_test[col].dtype=='object':
          #print(col)
          lbl = LabelEncoder()
          lbl.fit(list(X_train[col].values)+list(X_val[col].values) + list(X_test[col].values))
          X_train[col] = lbl.transform(list(X_train[col].values))
          X_val[col] = lbl.transform(list(X_val[col].values))
          X_test[col] = lbl.transform(list(X_test[col].values))

  return X_train,X_val ,X_test,lbl

# Ejemplo de uso:
# Suponiendo que X_train y X_test son tus DataFrames originales
X_train_final, X_val_final  ,X_test_final,label_encoders = label_encode_columns(X_train, X_val,X_test,cat_cols)



### Normalización y estandarización de las variables.

In [23]:
scaler_min_max = MinMaxScaler()
X_train_final[num_cols] = scaler_min_max.fit_transform(X_train_final[num_cols])
X_test_final[num_cols] = scaler_min_max.transform(X_test_final[num_cols])
X_val_final[num_cols] = scaler_min_max.transform(X_val_final[num_cols])



### Uso de la Validación Cruzada para Determinar el Número de Componentes en PCA


Seleccionamos una porción de los datos de entrenamiento para evaluar de manera óptima el modelo y realizar el análisis PCA.

In [24]:
# Definir la fracción del conjunto de datos para preentrenamiento
fraction = 0.5  # 10%


y = np.ravel(y_train)

# Dividir el conjunto de datos para obtener el subconjunto estratificado
X_pretrain, _, y_pretrain, _ = train_test_split(X_train_final, y, train_size=fraction, stratify=y, random_state=42)

# Verificar las proporciones de las clases
print("Proporción de clases en el conjunto original:", np.bincount(y) / len(y))
print("Proporción de clases en el subconjunto de preentrenamiento:", np.bincount(y_pretrain) / len(y_pretrain))
print(f"Tamaño del subconjunto de preentrenamiento: {len(y_pretrain)} muestras")

Proporción de clases en el conjunto original: [0.97999267 0.02000733]
Proporción de clases en el subconjunto de preentrenamiento: [0.97999259 0.02000741]
Tamaño del subconjunto de preentrenamiento: 121405 muestras


In [25]:
## Balanceo sin PCA
undersampler = RandomUnderSampler(random_state=42)
X_pretrain_B, y_pretrain_B,= undersampler.fit_resample(X_pretrain, y_pretrain)
# Mostrar la distribución de clases después del submuestreo
X_pretrain_B.shape, y_pretrain_B.shape

((4858, 184), (4858,))

In [26]:
unique, counts = np.unique(y_pretrain_B, return_counts=True)

# Crear un diccionario para mostrar los resultados
value_counts = dict(zip(unique, counts))
print(value_counts)

{0: 2429, 1: 2429}


In [27]:
# Inicia el cronómetro

# Número máximo de componentes principales (puede ser el número de características)
#max_components = X_pretrain.shape[1]
max_components = 50
# Inicializar la validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Inicializar el modelo de clasificación
classifier = RandomForestClassifier(random_state=42)

# Array para almacenar los scores de validación cruzada
cv_scores = []

# Realizar PCA y validación cruzada para diferentes números de componentes
for n_components in range(1, max_components + 1):
    #start_time = time.time()
    # Aplicar PCA

    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_pretrain_B)

    # Evaluar el modelo con validación cruzada
    scores = cross_val_score(classifier, X_pca, np.ravel(y_pretrain_B), cv=cv, scoring='accuracy')

    # Guardar el promedio de los scores
    cv_scores.append(scores.mean())
    # print(f'PCA con {n_components} componentes')
    # print(f'Accuracy promedio: {scores.mean():.4f} ± {scores.std():.4f}')
    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print(f"El tiempo de entrenamiento fue de {elapsed_time:.2f} segundos")

# Determinar el número óptimo de componentes
optimal_components = np.argmax(cv_scores) + 1
optimal_scores_mean = cv_scores[optimal_components - 1]  # Aquí se ajusta correctamente el índice
print(f'Número óptimo de componentes: {optimal_components}')
print(f'Score de validación cruzada con {optimal_components} componentes: {cv_scores[optimal_components - 1]}')
print(f'Accuracy promedio: {optimal_scores_mean:.4f}')

# Graficar los scores de validación cruzada
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(1, max_components + 1), cv_scores, marker='o', linestyle='--')
plt.xlabel('Número de Componentes')
plt.ylabel('Accuracy')
plt.title('Validación Cruzada para PCA y Clasificación Binaria')
plt.show()


KeyboardInterrupt: 

### Preprocessing con PCA

In [28]:
## Balanceo de X_train y y_train totales
undersampler = RandomUnderSampler(random_state=42)
X_train_B, y_train_B,= undersampler.fit_resample(X_train_final, y_train)
# Mostrar la distribución de clases después del submuestreo
X_train_B.shape, y_train_B.shape

((9716, 184), (9716,))

In [29]:
unique, counts = np.unique(y_train_B, return_counts=True)

# Crear un diccionario para mostrar los resultados
value_counts = dict(zip(unique, counts))
print(value_counts)

{0: 4858, 1: 4858}


In [30]:
## Balanceo y pca

pca = PCA(n_components=26)
X_train_pca_B = pca.fit_transform(X_train_B)



### Exportar preprocesamiento

In [31]:

dir_data_preprocessing='/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/preprocessing_objects'
ruta_archivo_encoder = os.path.join(dir_data_preprocessing, 'label_encoders.joblib')
ruta_archivo_scaler = os.path.join(dir_data_preprocessing, 'scaler_min_max.joblib')
ruta_archivo_pca = os.path.join(dir_data_preprocessing, 'pca.joblib')


joblib.dump(label_encoders, ruta_archivo_encoder)
joblib.dump(scaler_min_max,ruta_archivo_scaler)
joblib.dump(pca, ruta_archivo_pca)

['/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/preprocessing_objects/pca.joblib']




Se probó ajustando el porcentaje de valores permitidos por columna para maximizar la cantidad de columnas utilizadas en el análisis de componentes principales (PCA). Los resultados mostraron que el número máximo de componentes principales (PCA) obtenidos es de 4 cuando se permiten columnas con hasta un 20% de valores faltantes:

- Columnas con menos del 10% de valores faltantes: 3 componentes
- Columnas con menos del 20% de valores faltantes: 4 componentes
- Columnas con menos del 50% de valores faltantes: 4 componentes
- Columnas con menos del 60% de valores faltantes: 4 componentes

Se observó que la máxima varianza se logra con un número reducido de componentes principales, lo cual indica que gran parte de la información se puede capturar eficientemente con pocos componentes.

Por lo tanto vamos a trabajas con columnas que no tengan más del 20 de valores nulos.

In [32]:
# Guardar X y y en archivos CSV

#dir_data_processed = '../data/processed'
dir_data_processed = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/processed'
ruta_archivo_X_train = os.path.join(dir_data_processed, 'X__train_procesados.csv')
ruta_archivo_X_test = os.path.join(dir_data_processed, 'X_test_procesados.csv')
ruta_archivo_X_val = os.path.join(dir_data_processed, 'X_val_procesados.csv')

ruta_archivo_y_train = os.path.join(dir_data_processed, 'y_train_procesados.csv')
ruta_archivo_y_test = os.path.join(dir_data_processed, 'y_test_procesados.csv')
ruta_archivo_y_val = os.path.join(dir_data_processed, 'y_val_procesados.csv')


X_train_final.to_csv(ruta_archivo_X_train, index=False, encoding='utf-8')
X_val_final.to_csv(ruta_archivo_X_val, index=False, encoding='utf-8')
X_test_final.to_csv(ruta_archivo_X_test, index=False, encoding='utf-8')

y_train.to_csv(ruta_archivo_y_train, index=False, encoding='utf-8')
y_val.to_csv(ruta_archivo_y_val, index=False, encoding='utf-8')
y_test.to_csv(ruta_archivo_y_test, index=False, encoding='utf-8')



In [33]:
y_train.shape

(242811,)